# Hoare Logic
Separation logic. Not going to play that nice with z3.

Hmm. The raw cvc5 interface isn't that pleasant.

Winskel
Software Foundations
Dafny Book
Frama-C

Use Weepy?



    


In [11]:
from z3 import *

# Yeah. Is there a point to this? Should I just use python datatypes?

Imp = Datatype("Imp")
Imp.declare("skip")
Imp.declare("seq", ("left", Imp), ("right", Imp))
Imp.declare("assign", ("var", StringSort()), ("expr", IntSort())) # it is eaisest if expr is a z3 expr.
Imp = Imp.create()
Imp.assign(StringVal("x"), 1)# IntVal(1))
x = StringVal("x")
prog = Imp.seq(Imp.assign(x, 1), 
               Imp.assign(x, 2))

# This doesn't work (?) because I need to also embed the logic in itself. Meh.
# or   ArraySort(Rho,BoolSort()) # i did this in a pure smtlib example once.
# Oh actually this does play in with quote.


#Triple = Datatype("Triple")
#Triple.declare("triple", ("pre", IntSort()), ("cmd", Imp), ("post", IntSort()))
from collections import namedtuple
Triple = namedtuple("Triple", ["pre", "cmd", "post", "proof"])
# or two datatypes Triple and Proof(triple,pf)


# wp as a hoare axiom schema.
def wp(cmd, post) -> Triple:
    def wp(cmd, post):
        if cmd.sort().name() == "assign":
            return post.substitute(cmd.var, cmd.expr)
        elif cmd.sort().name() == "seq":
            return wp(cmd.left, wp(cmd.right, post))
        elif cmd.sort().name() == "skip":
            return post
        else:
            raise Exception("Unknown command: %s" % cmd)
    return Triple(wp(cmd,post), cmd, post, "wp_ax")

def seq(trip1 : Triple, trip2: Triple, by=[]) -> Triple:
    pre1, cmd1, post1 = trip1
    pre2, cmd2, post2 = trip2.cmd
    prove(Imp(And(by, post1), pre2))
    return Triple(pre1, Imp.seq(cmd1,cmd2), post2)

def while_(inv, body:Triple, by=[]):
    pass



# hmm. Maybe an tree deconstruction of different kinds might lead to different proof systems of cfg systems.


In [ ]:
import cvc5
solver = cvc5.Solver()
solver.setLogic("ALL")
realSort = solver.getRealSort()
constraint1 = solver.mkTerm(Kind.LT, zero, x)
solver.assertFormula(constraint4)
solver.resetAssertions()
r2 = solver.checkSat()
unsatCore = solver.getUnsatCore();
# https://greentreesnakes.readthedocs.io/en/latest/nodes.html 
import ast
tree = ast.parse("x = x + 3", mode="single")
match tree.body[0]:
    case ast.Assign([ast.Name(name, ast.Store())], expr):
        print(name, expr)
    case x:
        print(x)


def wp(stmt,post):
    match stmt:
        case ast.Assign([ast.Name(name, ast.Store())], expr):
            return z3.substitute(z3.Int("name"), expr)
        case ast.Pass():
            return post
        case _:
            assert False

def triple(pre,stmt,post):
    mywp = wp(stmt,post)

def assign_triple(pre : Form, name, expr, post : Form, by=[]):
    wp = z3.substitute(post, (x, expr))
    check(infer(by, pre > wp))
    return trust( (pre, ast.Assign([name], expr), post))

def seq_triple(s1 : Triple, s2: Triple):